# Hent bilder for Harriet Backer (eller andre)

Bruker iiif-manifest, som gir alle bildene, sammen med search for bokhylla. Dataene fra den her notebooken er lagret til en fil (se nedenfor) så om det ikke endres noe kan dataene hentes fra den.

In [10]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import pandas as pd
import json

In [11]:
mu.css()

In [12]:
mu.update('nbpictures', silent=True)
import nbpictures as pic

In [16]:
import requests

def super_search(term = '', creator = '', number = 50, page = 0, mediatype = 'bilder'):
    """Søk etter term og få ut json"""
    
    number = min(number, 50)
    
    filters = []
    
    params = {
        'page':page, 
        'size':number
    }
    
    if creator != '':
        filters.append('creator:{c}'.format(c=creator))
    
    if mediatype != '':
        filters.append('mediatype:{mediatype}'.format(mediatype=mediatype))
    
    if filters != []:
        params['filter'] = filters
    
    if term != '':
        params['q'] = term
    
    r = requests.get("https://api.nb.no:443/catalog/v1/items", params = params)
    return r.json()

def find_urns_sesam(term = '', creator = '', number=50, page=0, mediatype='bilder'):
    """generates urls from super_search for pictures"""
    x = super_search(term = term, creator = creator, number = number, page = page, mediatype=mediatype)
    try:
        sesamid =[
            f['id']
            for f in x['_embedded']['items'] 
            if f['accessInfo']['accessAllowedFrom'] == 'EVERYWHERE'
            and 'thumbnail_custom' in f['_links']
        ]
    except:
        sesamid = []
    return sesamid

In [19]:
find_urns_sesam('',creator='hamsun', page=10, mediatype='privatarkivmateriale')

['d10a8b7e85766514b28cddc1d060fb33',
 '8c21355573e66a0c3d04d7e9f8bbd759',
 '30ee75168624a73624924202716403d3',
 'aa5fde2b3322229037f390b2f0f0a402',
 'd1a7d17240532df7733e70edbd8989c7',
 'd51df917bb849f9356e695c4bf878136',
 '789720e43c0589e9a7d11e194c6a67b1',
 'b23566895e8a430c8dc42484a11ac74e',
 '04d8a302a9d7dd3e469c4eae762d554d']

Hent referanse til alle bildene. Funksjonen `find_urls` henter URLer. Fra dem peller vi ut URNene, og gir til `iiif_manifest`, som i sin tur gir alle bildene.

In [33]:
urls = pic.find_urls('Harriet Backer', page=0, mediatype='privatarkivmateriale')
for i in range(1,100):
    t = pic.find_urls('Harriet Backer', page = i, mediatype = 'privatarkivmateriale')
    if t == []:
        print(i)
        break
    else:
        urls += t


14


Så hentes URNene fra listen over URLer

In [40]:
urns = ['_'.join(url.split('/')[6].split('_')[:-1]) for url in urls]

In [42]:
urns[:3]

['URN:NBN:no-nb_digimanus_326898',
 'URN:NBN:no-nb_digimanus_81631',
 'URN:NBN:no-nb_digimanus_321355']

Med `iiif_manifest` kommer metadata.

In [70]:
pages = dict()
for urn in urns:
    x = pic.iiif_manifest(urn)
    pages[urn] = [page['images'][0]['resource']['@id'] for page in x['sequences'][0]['canvases']]

Lagre bildene til fil som en json-struktur

In [87]:
with open('bilder.json', 'w') as fp:
    json.dump(pages, fp)

## Filtrering av data med metadata

For videre filtrering kan`iiif_manifest` kjøres på nøklene i bildefilen. Så her er `bilder` akkurat det samme som `pages`.

In [89]:
with open('bilder.json') as fp:
    bilder = json.load(fp)

In [14]:
pic.iiif_manifest('f3b2df9cff5e6d1ab6e98380946d5561')

{'@context': 'http://iiif.io/api/presentation/2/context.json',
 '@type': 'sc:Manifest',
 '@id': 'https://api.nb.no/catalog/v1/iiif/f3b2df9cff5e6d1ab6e98380946d5561/manifest{?fields,profile}',
 'label': 'Brevs. 130 Brev fra Hamsun, Knut (1859-1952) til Vetlesen, Thom (1856-1933)',
 'metadata': [{'label': 'Tilgang',
   'value': 'Kan kun åpnes i Nasjonalbibliotekets lokaler i Oslo'},
  {'label': 'Tittel',
   'value': 'Brevs. 130 Brev fra Hamsun, Knut (1859-1952) til Vetlesen, Thom (1856-1933)'},
  {'label': 'Dato', 'value': '26.11.1920'},
  {'label': 'Kilde for metadata', 'value': 'hanske.nb.no (51084)'},
  {'label': 'Medietype', 'value': 'Brev og manuskripter'},
  {'label': '',
   'value': '<a href="https://www.nb.no/items/f3b2df9cff5e6d1ab6e98380946d5561?manifest=https://api.nb.no/catalog/v1/iiif/f3b2df9cff5e6d1ab6e98380946d5561/manifest" target="_blank"><img src="https://www.nb.no/content/uploads/2018/08/logo-iiif.png" alt="IIIF Drag-n-drop"></a>'}],
 'license': 'https://www.nb.no/lise

In [100]:
pic.get_picture_from_url(bilder['URN:NBN:no-nb_digimanus_326898'][0]).save('t.jpg')